In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open('../styles/custom.css', 'r').read()
    return HTML(styles)
css_styling()

# Pressure coefficient and force

We will apply the vortex panel method developed in the last section to determine the pressure coefficient in the flow and the pressure force on a body.

## VortexPanel module

I've put all the important functions from the previous notebooks in the VortexPanel module. Let's import and test them:

In [ ]:
import numpy
from matplotlib import pyplot
%matplotlib inline
from VortexPanel import make_circle, solve_gamma, plot_flow, flow_velocity

In [ ]:
help(solve_gamma)

We can test them by solving for the flow around a circle again.

In [ ]:
N = 33
circle = make_circle(N)         # set-up geom
solve_gamma(circle)             # find gamma
plot_flow(circle)               # plot the flow

Looks good.

### Bernoulli equation

Next we can determine the pressure $p$ in the flow using the Bernoulli equation.

$$ p+\frac 12 \rho u^2-\rho gz = B $$

where $\rho$ is the density, $\rho gz$ is the head, and $B$ is the Bernoulli coefficient. 

We will assume no hydrostatic pressure.

##### Quiz 1

What is the pressure far from the body, $p_\infty$?

1. $p_\infty = \frac 12 \rho U_\infty^2$
1. $p_\infty = 0$ 
1. $p_\infty = B-\frac 12 \rho U_\infty^2$ 

### Pressure coefficient

To avoid the ambiguity of the Bernoulli coefficient, we typically work with the pressure coefficient $c_p$:

$$c_p = \frac{p-p_\infty}{\frac 12\rho U_\infty^2}$$

##### Quiz 2

How can we simplify the pressure coefficient when there is no hydrostatic pressure?

1. $c_p = 1-|u|^2/U_\infty^2$
1. $c_p = 0$
1. $c_p = 1-p/p_\infty$

---
We can use this equation to plot the pressure coefficient in the flow.

In [ ]:
def field_c_p(u,v): return 1-(u**2+v**2)
plot_flow(circle,contour_fun=field_c_p,
          contour_min=None,contour_lab='Pressure coefficient')

##### Quiz 3

What is $c_p$ at the stagnation points?

1. $c_p = 0$
1. $c_p = 1$
1. $c_p = \infty$

## Surface pressure

The pressure on the surface of the body relates directly to the forces. Since we're using a vortex panel method, this mean we want the pressure on each panel.

##### Quiz 4

What is the velocity magnitude $|u_e|$ just on the **flow side** of panel $i$? 

1. 0
1. $\left|\vec U +\sum_{j=0}^{N-1} \gamma_j \vec f_j(x_i,y_i)\right|$
1. $\left|\gamma_i\right|$

Hint: Remember that we have set a boundary condition on the body side of the panel.

---

This means we have a very simply equation for the surface pressure when using the vortex panel method! However, there is a problem with how the data is organised. 

##### Numerical Fundamental: Data representation
##### Put your data in a convenient form before processing

It would be most convenient for post-processing if we had an array of `gamma` - but we don't. Right now we have an array of `Panel`s, each with their own value of $\gamma$. 

However, we can make a simple function to get any attribute of `Panel` into an array of its own:

In [ ]:
def get_array(panels,key):
    return numpy.array([getattr(p,key) for p in panels])

circle_yc = get_array(circle,'yc')
print(circle_yc)

This function uses a [list comprehension](http://www.pythonforbeginners.com/lists/list-comprehensions-in-python/) to make a list of the attribute you want for each panel, and then converts it into an easy to use numpy array.

__Note__: you pass `get_array` the name of the attribute you want _as a string_ (inside single or double quotes). 

---

## Pressure force

Once we have the surface pressure we can use it to determine the pressure force on the body, as

$$ \vec F_p = -\oint p \hat n da $$

where $\oint da$ is the integral over the body surface, and $\hat n$ is the normal vector to the surface. 

The pressure drag coefficient $C_D$ is then:

$$ C_D = \frac{\vec F_p \cdot \hat x} { \tfrac 12 \rho U_\infty^2 A }  = \frac{-\oint p n_x da}{ \tfrac 12 \rho U_\infty^2 A } $$

where $A$ is the cross-sectional area (in 2D this is the thickness $t$). 

Note that we need the normal vector to compute this integral...

In [ ]:
get_array(circle,'nx')

`'Panel' object has no attribute 'nx'`. Oops... 

##### Quiz 5

How can we determine the normal $\hat n$ from the saved attributes of a panel?

1. $\hat n = \vec x_1-\vec x_0$
1. $n_x,n_y = -s_y,s_x$
1. $\hat n = \hat z \times \hat s$

Hint: 
![image](resources/graphics6.png)

---


##### Your turn #3

 - **Write** the expression for $c_p$ in terms of $\gamma$.
 - **Complete** the c_p function to evaluate this expression for the vortex panel method.
 - **Plot** $c_p$ vs the distance around the circle, $s$, using this function and...  
 - **Discuss** the accuracy of the numerical method compared to the analytic solution that I've plotted for you $$ c_p = 1-4\sin^2(s/R) $$ where $R=1$ is the circle radius.
 - **Write** the expression for the pressure drag coefficient $C_D$ in terms of $c_p$ and the panel attributes (like $s_y$). You'll find the following identity helpful in simplifying the expression: $$ \oint k \hat n ds = 0 $$ where $k$ is any constant.
 - **Complete** the `C_D` function using the expression above. Numerically evaluate the integral as a `sum` over the panels.
 - **Discuss** the drag on the circle. Does the numerical method match what you expect analytically?

---

##### Solution #3

$c_p = ?? $

In [ ]:
def c_p(gamma):
    
    ### your code for c_p here

    return c_p

In [ ]:
N=32
circle = make_circle(N)      # set-up geom
solve_gamma(circle)          # determine pflow solution

s = 2*numpy.pi*numpy.linspace(0.5/N,1-0.5/N,N)
pyplot.plot(s,1-4*numpy.sin(s)**2, label='analytic')

# your code here to plot the numerical result

pyplot.legend(loc='upper right')
pyplot.xlabel(r'$s/R$', fontsize=18)
pyplot.ylabel(r'$c_p$', fontsize=18)

$C_D =  ??$

In [ ]:
def C_D( panels ):
    # get attribute arrays
    gamma = get_array(panels,'gamma')
    yc = get_array(panels,'yc')
    # your code for other arrays

    # get the thickness
    t = max(yc)-min(yc)
    
    # your code to compute the drag coefficient using `sum` over panel arrays
    return C_D

print(C_D(circle))